In [2]:
# importanto as bibliotecas necessárias
import os
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import (ensemble, preprocessing, tree)
from sklearn.metrics import(auc, confusion_matrix, roc_auc_score,roc_curve)
from sklearn.model_selection import (train_test_split, StratifiedKFold)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from yellowbrick.classifier import ConfusionMatrix #vou chamar quando for usar. Rolou um BO no início. Pois, precisei comendar todas as bibliotecas do yellowbrick até realmente usar
from yellowbrick.classifier.rocauc import roc_auc
from yellowbrick.model_selection import LearningCurve

# Bibliotecas adicionadas durante o coding
#import pandas_profiling

In [3]:
# carregando o dataset
try:
	df = pd.read_csv('https://raw.githubusercontent.com/pandas-dev/pandas/master/doc/data/titanic.csv')
except:
	# Alternative: use seaborn's built-in titanic dataset
	df = pd.read_csv('https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv')
orig_df = df

print(df.shape)
# pclas: classe do passageiro (1 = primeira classe, 2 = segunda classe, 3 = terceira classe)
# survived: 0 = No, 1 = Yes
# name: nome do passageiro
# sex: sexo do passageiro
# age: idade do passageiro em anos
# sibsp: número de irmãos/cônjuges a bordo
# parch: número de pais/filhos a bordo
# ticket: número do bilhete
# fare: tarifa paga pelo passageiro
# cabin: número da cabine
# embarked: porto de embarque (C = Cherbourg, Q = Queenstown, S = Southampton)
# df.boat: bote salva-vida

# organizando o dataset
dir_path='data/'
try:
	# Vou criar uma pasta para conter todas as tabelas e organizar melhor o projeto
	# exist_ok=True      Evita um erro caso o diretório já exista
    os.makedirs('data', exist_ok=True) 
    os.makedirs('data/processed', exist_ok=True)
    os.makedirs('data/preprocessed', exist_ok=True)
    # Salvar o orig_df como um arquivo csv na pasta data
    orig_df.to_csv(os.path.join(dir_path, 'orig_df.csv'), index=False) #tbm faremos o mesmo quando o df for limpo ou fracionado
except Exception as dir_path:
    print(f'Erro ao criar o diretório: {dir_path} e seus itens')

(891, 12)


# Estudo de dados para limpeza dos dados
Aqui queremos trabalho sobre os NaNs ou outliers ou dados inconsistentes. Assim como queremos ermover colunas que possam gerar algum data leakege. Tbm vamos criar colunas dummy ('pd.get_dummies')caso precisemos.
> Data leakage ocorre quando informações do conjunto de dados de teste ou validação vazam para o conjunto de treinamento durante o pré-processamento ou modelagem, ou quando informações do target vazam para as features. 

Nesse processo vamos utilizar duas bibliotecas do pandas 
- pandas_profiling
- pandas_profiling.ProfileReport(df)
Para gerar relatórios em notebooks para vermos detalhes estatísticos descritivos e dos quantis, além de histogramas

In [4]:
# exibindo as primeiras linhas do dataset
print(df.head())
print(df.shape)
print(f'nossas colunas são: {df.dtypes}')   # Estudamos os tipos de colunas que possuímos

# Gerar relatório de perfil (tenta ydata_profiling, senão usa pandas_profiling)
# Não importamos `display` diretamente aqui para evitar import errors com IPython.core.display
try:
    from ydata_profiling import ProfileReport
except ImportError:
    try:
        from pandas_profiling import ProfileReport
    except ImportError as e:
        raise ImportError(
            "Instale 'ydata-profiling' (recomendado) ou 'pandas-profiling' antes de gerar o relatório."
        ) from e

report = ProfileReport(df)
# Mostrar no notebook (iframe) - to_notebook_iframe() já cuida da exibição no Jupyter
try:
    report.to_notebook_iframe()
except Exception:
    # Caso a exibição inline falhe, escrevemos o arquivo HTML e informamos o caminho
    report.to_file(os.path.join(dir_path, "preprocessed", "profile_report.html"))
    print(f"Relatório salvo em {os.path.join(dir_path, 'preprocessed', 'profile_report.html')}")
else:
    # também salvar em HTML
    report.to_file(os.path.join(dir_path, "preprocessed", "profile_report.html"))

# analizando o df de forma truncada as duas colunas iniciais
df.describe().iloc[:, :2]  # Tanto `.loc` quanto `.iloc` são atributos essenciais dos DataFrames do Pandas e ambos são usados ​​para selecionar subconjuntos específicos de dados. Seu propósito é acessar e permitir a manipulação de uma parte específica do DataFrame, em vez do DataFrame inteiro.

# vamos utilizar o isnull para verificar valores nulos
df.isnull().sum()
df.isnull().mean()

# analise de algumas das linhas com dados ausentes com boleanos
mask = df.isnull().any(axis=1)
print(mask.sum(), "linhas com pelo menos um NaN")
display(df[mask].head())           # inspeciona as linhas com NaN
print(df.isnull().sum()[lambda s: s>0])  # colunas com NaNs e suas contagens
print((df.isnull().mean()*100).sort_values(ascending=False).head())  # % de NaNs por coluna
#mask.head() #linhas
#df[mask].body.head()



   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                              Heikkinen, Miss Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
(8

/home/tiago-ubunto/Documentos/Estudo/ti_estudo/meus-projetos/Titanic/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 56.12it/s]

Relatório salvo em data/preprocessed/profile_report.html
708 linhas com pelo menos um NaN


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,3,1,3,"Heikkinen, Miss Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,8,0,3,"Palsson, Master Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S


Age         177
Cabin       687
Embarked      2
dtype: int64
Cabin          77.104377
Age            19.865320
Embarked        0.224467
PassengerId     0.000000
Name            0.000000
dtype: float64


In [5]:
# categorização para limpeza
# defina o target antes de remover a coluna
print(df.dtypes)
y = df['Survived']   #Estou em dúvida se criaria outra coluna "dead" como target alternativo. Se pensarmos em Boleanos, seriam antônimos
X = df.drop(columns="Survived")
print(df.dtypes)

if 'Name' in df.columns:
    name= df['Name']
    print(f'a cabeça do nosso objeto names: {name.head(3)}')
else:
    print("A coluna do objeto Name está presente somente no dataframe original.")
#removendo colunas que não serão usadas ou podem gerar viés no modelo


df=df.drop(columns=["Name","Ticket", "Survived", "Cabin"]) 
# logo vamos criar uma coluna "dummy"
df=pd.get_dummies(df)  # ou # df=pd.get_dummies(df,drop_first=True) #para evitar a dummy trap... No caso as colunas male_sex e famale são inversos perfeitos
df=df.drop(columns=["Sex_female"])  # removendo uma das colunas dummies para evitar dummy trap
print(f'Nossas colunas são com os dummies aplicados: {df.columns}') 


PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object
a cabeça do nosso objeto names: 0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                Heikkinen, Miss Laina
Name: Name, dtype: object
Nossas colunas são com os dummies aplicados: Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_male',
       'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')


In [6]:
# Separação de amostra #amostra_para_validação_do_modelo
# scikit-learn para separar 30% para os testes
# import library q usaremos abaixo
from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=42)

#Impotação dos dados
##como a coluna idade tem valores ausentes. Vamos imputar udade a partir dos valores numéricos apenas no conjunto de treinamento, e então

In [7]:
# IterativeImputer é experimental em algumas versões do sklearn:
# importe o "enable" antes de importar o estimador
from sklearn.experimental import enable_iterative_imputer  # deve vir antes DO DE BAIXO
from sklearn.impute import IterativeImputer

# Colunas numéricas a imputar (use nomes como strings)
num_cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

# verificar quais colunas realmente existem nos dataframes (evita KeyError)
num_cols = [c for c in num_cols if c in X_train.columns]

imputer = IterativeImputer(random_state=0)
X_train.loc[:, num_cols] = imputer.fit_transform(X_train[num_cols])
X_test.loc[:, num_cols] = imputer.transform(X_test[num_cols])

# preencher possíveis NaNs restantes com a mediana do conjunto de treino (aplicado às colunas numéricas)
meds = X_train[num_cols].median()
X_train.loc[:, num_cols] = X_train[num_cols].fillna(meds)
X_test.loc[:, num_cols] = X_test[num_cols].fillna(meds)

In [8]:
#normalização dos dados
## este processo será importante para nosso modelo ter uma melhor performance
cols=["Pclass","Age","Sibsp","Fare"]
sca=preprocessing.StandardScaler()
X_train=sca.fit_transform(X_train)
X_train=pd.DataFrame(X_train, columns=X.columns)
X_test=sca.transform(X_test)
X_test=pd.DataFrame(X_test, columns=cols)

ValueError: could not convert string to float: 'Dodge, Master Washington'

In [ ]:
#Refatoração
## A medida que o ML for evoluindo, vamos precisar realizar ajustes e até mesmo import o original dataset novamente para refinar o modelo.
## Faremos duas funções para o processo
def tweak_titanic(df):
    df=df.drop(columns=["Name","Ticket","Survived","Cabin"]).pipe(pd.get_dummies, drop_first=True)
return df

def get_train_test_X_y(df, y_col, size=0.3, std_cols=None):
    y = df[y_col]
    X= df.drop(columns=y_col)
    X_train, X_test, y_train, y_test= model_selection.train_test_split( X, y, test_size=size, random_state=42)
    cols=X.columnsnum_cols=["Pclass", "Age", "Sibsp", "Parch", "Fare"]
    fi=impute.IterativeImputer()
    X_train.loc[:, num_cols]= fi.fit_transform(X_train[num_cols])
    X_test.loc[:, num_cols] = fi.transform( X_test[num_cols])

    if std_cols:
        std=preprocessing.StandardScaler()
        X_train.loc[:, std_cols] = std.fit_transform(X_train[std_cols])
        X_test.loc[:, std_cols] = std.transform(X_test[std_cols])
return X_train, X_test, y_train, y_test

ti_df = tweaks_tweak_titanic(orig_df)
std_cols="Pclass,Age,Sibsp,Fare".split(",")
X_train, X_test, y_train, y_test= get_train_test_X_y(ti_df, "Survived", std_cols=std_cols)

In [ ]:
#modelo_de_base
#uma_base_realmente_simples_que_o_modelo_usara_como_comparador

from sklearn.dummy import DummyClassifier
bm = DummyClassifier()
bm.fit(X_train, y_train)
bm.score(X_test, y_test) #nos_dará_nossa_precisao

from sklearn import metrics
metrics.precision_score(y_test, bm.predict(X_test))



#familias
#Neste_projeto_vamos_comparar_pontuações_AUC_e_o_desvio_padão_usando_a_validação_cruzadak_fold
X=pd.concat([X_train, X_test])
y=pd.concat([y_train, y_test])

from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier 
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

for model in [DummyClassifier, LogisticRegression, DecisionTreeClassifier, KNeighborsClassifier, GaussianNB, SVC, RandomForestClassifier, XGBClassifier]: 
    cls = model()
    kfold = model_selection.KFold(n_splits=10, random_state=42, shuffle=True)
    s = cross_val_score(cls, X, y, scoring="roc_auc", cv=kfold)
    print(f'{model.__name__:22} ; AUC: {s.mean():.3f} STD: {s.std():.2f}') #Por que usar o AUC em vez da Acurácia? A acurácia pode ser enganosa se os seus dados estiverem desbalanceados.
   



NameError: name 'X_train' is not defined

In [ ]:
# Criando o modulo
## Usaremos um classificador de random forest
rf=ensemble.RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

In [ ]:
#Avaliação de modelo
rf.score(X_test, y_test)
metrics.precision_score(y_test, rf.predict(X_test))

for col, val in sorted(
zip(X_train.columns, rf.feature_importances_,), key=lambda x: x[1], reverse=True,)[:5]):
    print(f'{col:10}{val:10.3f}')

In [ ]:
#Otimizando o modelo
